In [1]:
! pip install tensorflow keras --upgrade

In [2]:
import tensorflow as tf
import keras
print(tf.__version__)
print(keras.__version__)

2.7.0
2.7.0


In [3]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.applications.resnet_v2 import ResNet152V2

In [4]:
#from google.colab import drive
#drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [4]:
data_dir = "train"
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(256, 256),
    batch_size=32)

Found 47332 files belonging to 325 classes.
Using 37866 files for training.


In [17]:
data_dir = "valid"
valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(256, 256),
    batch_size=32)

Found 1625 files belonging to 325 classes.
Using 1300 files for training.


In [18]:
data_dir = "test"
valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(256, 256),
    batch_size=32)

Found 1625 files belonging to 325 classes.
Using 1300 files for training.


In [6]:
class_names = train_ds.class_names
print(class_names)

['AFRICAN CROWNED CRANE', 'AFRICAN FIREFINCH', 'ALBATROSS', 'ALEXANDRINE PARAKEET', 'AMERICAN AVOCET', 'AMERICAN BITTERN', 'AMERICAN COOT', 'AMERICAN GOLDFINCH', 'AMERICAN KESTREL', 'AMERICAN PIPIT', 'AMERICAN REDSTART', 'ANHINGA', 'ANNAS HUMMINGBIRD', 'ANTBIRD', 'ARARIPE MANAKIN', 'ASIAN CRESTED IBIS', 'BALD EAGLE', 'BALD IBIS', 'BALI STARLING', 'BALTIMORE ORIOLE', 'BANANAQUIT', 'BANDED BROADBILL', 'BANDED PITA', 'BAR-TAILED GODWIT', 'BARN OWL', 'BARN SWALLOW', 'BARRED PUFFBIRD', 'BAY-BREASTED WARBLER', 'BEARDED BARBET', 'BEARDED BELLBIRD', 'BEARDED REEDLING', 'BELTED KINGFISHER', 'BIRD OF PARADISE', 'BLACK & YELLOW bROADBILL', 'BLACK BAZA', 'BLACK FRANCOLIN', 'BLACK SKIMMER', 'BLACK SWAN', 'BLACK TAIL CRAKE', 'BLACK THROATED BUSHTIT', 'BLACK THROATED WARBLER', 'BLACK VULTURE', 'BLACK-CAPPED CHICKADEE', 'BLACK-NECKED GREBE', 'BLACK-THROATED SPARROW', 'BLACKBURNIAM WARBLER', 'BLONDE CRESTED WOODPECKER', 'BLUE COAU', 'BLUE GROUSE', 'BLUE HERON', 'BLUE THROATED TOUCANET', 'BOBOLINK', 'BO

In [7]:
num_classes = len(class_names)
preprocess_input_resnet_v2 = tf.keras.applications.resnet_v2.preprocess_input

base_model = tf.keras.applications.ResNet152V2(input_shape=(256, 256, 3),
                                               include_top=False,
                                               weights='imagenet')


base_model.trainable = False

image_batch, label_batch = next(iter(train_ds))
feature_batch = base_model(image_batch)

#global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
#feature_batch_average = global_average_layer(feature_batch)
flatten_layer = tf.keras.layers.Flatten()
feature_batch_flatten = flatten_layer(feature_batch)
prediction_layer = tf.keras.layers.Dense(num_classes, kernel_regularizer=tf.keras.regularizers.l2(0.001))
#prediction_batch = prediction_layer(feature_batch_average)
prediction_batch = prediction_layer(feature_batch_flatten)
inputs = tf.keras.Input(shape=(256, 256, 3))
#x = data_augmentation(inputs)
x = preprocess_input_resnet_v2(inputs)
x = base_model(x, training=False)
#x = global_average_layer(x)
x = flatten_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

#################################################
# COMPILE THE MODEL
##################################################
#
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [8]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=10,
    callbacks=[tensorboard_callback]
)

D:\Programming\Python\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/10
1184/1184 [==============================] - 5746s 5s/step - loss: 23.2993 - accuracy: 0.6261 - val_loss: 20.6176 - val_accuracy: 0.8200
Epoch 2/10
1184/1184 [==============================] - 5294s 4s/step - loss: 15.6983 - accuracy: 0.8989 - val_loss: 20.8135 - val_accuracy: 0.8415
Epoch 3/10
1184/1184 [==============================] - 5272s 4s/step - loss: 16.1729 - accuracy: 0.9132 - val_loss: 24.1724 - val_accuracy: 0.8292
Epoch 4/10
1184/1184 [==============================] - 5366s 5s/step - loss: 18.7738 - accuracy: 0.9060 - val_loss: 24.6941 - val_accuracy: 0.8508
Epoch 5/10
1184/1184 [==============================] - 5326s 4s/step - loss: 19.5241 - accuracy: 0.9188 - val_loss: 26.5221 - val_accuracy: 0.8469
Epoch 6/10
1184/1184 [==============================] - 5494s 5s/step - loss: 19.2628 - accuracy: 0.9245 - val_loss: 27.0384 - val_accuracy: 0.8523
Epoch 7/10
1184/1184 [==============================] - 5496s 5s/step - loss: 19.6911 - accuracy: 0.9257 - val_l

In [10]:
from keras.models import load_model
model.save('my_model.h5')

In [11]:
!pip install tensorboard
%load_ext tensorboard

In [12]:
import os
#logs_base_dir = DATA_ROOT / 'logs'  #создаем папку для хранения логов экспериментов
#print(logs_base_dir)
os.makedirs(log_dir, exist_ok=True)
%tensorboard --logdir {'"%s"' % log_dir}

In [13]:
def predict_bird(path, class_names):
  img = tf.keras.preprocessing.image.load_img(
    path, target_size=(256, 256, 3))
  img_array = tf.keras.preprocessing.image.img_to_array(img)
  return (class_names[np.argmax(model.predict(img_array.reshape(1, 256, 256, 3)))])

In [14]:
predict_bird('test/AMERICAN REDSTART/3.jpg', class_names)

'AMERICAN REDSTART'

In [19]:
class_names = train_ds.class_names
path = 'test/'
true_positiv = 0
for class_i in class_names:
  for i in range(1,6):
    path_bird = path + class_i
    path_bird += '/' + str(i) + '.jpg'
    prediction = predict_bird(path_bird, class_names)
    if prediction == class_i:
      true_positiv += 1
print(str(true_positiv / 1625 * 100))

87.13846153846154
